In [ ]:
# 1번 2번  XGBClassifier() 생성
import lightgbm as lgb
model = lgb.LGBMClassifier(
    objective = 'binary',
    boosting_type = 'gbdt',
    verbosity = -1,
    metric = 'binary_logloss'  
)
param_grid = {
    'max_depth' : [3,4,5,6],
    'num_leaves' : [7,15,31,63],
    'learning_rate' : [0.001,0.01,0.1],
    'bagging_fraction' : [0.4,0.6,0.8],
    'feature_fraction' : [0.4,0.6,0.8],
    'reg_alpha' : [0.9,1.1],
    'reg_lambda' : [0.9,1.1],
    'max_bin' : [100,200,300]
}
# 3번  KFold 지정
kfold = KFold(n_splits=6,random_state=1)

# 4번  GridSearchCV로 모델 생성
gsearch = GridSearchCV(model, param_grid=param_grid, cv=kfold, scoring='accuracy', n_jobs=4)

# 5번    
gsearch.fit(x,y)
print('final params', gcv.best_params_)
print('best score', gcv.best_score_)

# 6번  최적의 파라미터로 lightgbm 모델 생성
model= gsearch.best_estimator_

# 7번 
kfold = KFold(n_splits=6, random_state=24, shuffle=True)

# 8번
oof_result = np.zeros((len(test_x),6))
for idx, (train_idx, val_idx) in enumerate(kfold.split(train)):
    print('predict with KFold')
    train_x,train_y = x.iloc[train_idx],y.iloc[train_idx]
    valid_x,valid_y = x.iloc[val_idx],y.iloc[val_idx]

# 8-1번  각각의 데이터로 xgboost모델로 학습.
    model.fit(train_x,train_y,eval_set=[(valid_x,valid_y)],early_stopping_rounds=100,verbose=100)

# 8-2번  예측합니다.
    oof_result[:,idx] = model.predict(test_x)

# 9 
result = oof_result.mean(axis=1)
result_final = (result>0.5).astype(int)
accuracy_score(result_final,test_y)